In [1]:
# Import necessary Python, sklearn and/or tensorflow/keras modules
import os
import xmltodict
import cv2
import numpy as np
from keras.preprocessing import image
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

from keras.models import Sequential
from keras.layers import *

print("tf Version: ", tf.__version__)
print("Eager Execution mode: ", tf.executing_eagerly())

Using TensorFlow backend.


tf Version:  2.0.0
Eager Execution mode:  True


In [2]:
# Load the data
entries = range(3383)
X = np.zeros((3383,150,150,3)) # initialize array of zeros to avoid mem reallocation
y = []
categories = os.listdir("./Images/")
i = 0
for breed in categories:
  images = os.listdir("./Images/{}".format(breed))
  counter = 0
  for doggo in images:
    if counter == len(images)//6:
        break
    #print(len(X))
    #x=cv2.imread('Images/%s/%s'%(breed,doggo))
    #x=cv2.resize(x, (150,150))
    x = image.img_to_array(image.load_img('Images/%s/%s'%(breed,doggo), target_size=(150,150)))
    #x.reshape((1,) + x.shape) 
    #print(x.shape)
    X[i] = x
    with open("Annotation/%s/%s"%(breed,doggo.split('.')[0])) as annotation:
      annotation = xmltodict.parse(annotation.read())
      label = {'binding_box': [],\
               'breed': []}
      if type(annotation['annotation']['object']) is list:
        # If it is a list, there are multiple doggos in this image
        # and we need to keep track of all of them for regression
        for obj in annotation['annotation']['object']:
          label['breed'].append(obj['name'])
          label['binding_box'].append(obj['bndbox'])
      else:
        label['breed'].append(annotation['annotation']['object']['name'])
        label['binding_box'].append(annotation['annotation']['object']['bndbox'])
      label['image_dimensions'] = annotation['annotation']['size']
      y.append(label['breed'])
    #print(image.img_to_array(i).shape)
    i += 1
    if i % 100 == 0:
        print(i, end=' ')
    counter +=1

# Get data shape via built-in methods of sklearn, pandas or tensorflow/keras
print()
print(X.shape)
print(np.array(y).shape)

100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 
(3383, 150, 150, 3)
(3383,)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42 )

In [4]:
classifier = Sequential()
classifier.add(Conv2D(240, kernel_size = 3, input_shape = (150, 150, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3,3)))
classifier.add(Flatten())
classifier.add(Dense(120, activation = 'relu'))
classifier.add(Dense(120, activation = 'softmax'))

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [6]:
classifier.fit(X_train, y_train, batch_size = 20, epochs = 50, validation_split = 0.2)

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 2706 arrays: [array([['basset']], dtype='<U6'), array([['redbone']], dtype='<U7'), array([['Irish_water_spaniel']], dtype='<U19'), array([['Scottish_deerhound']], dtype='<U18'), array([['Irish_wolfhound']], dtype=...